In [76]:
import json
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry import shape
import pandas as pd

In [77]:
precincts=json.load(open('first-phase/Boston/Boston Police Districts.geojson','r'))
# precincts=precincts['features']
# print(len(precincts))

polygons=[]
# precincts[0]
cx,cy=0,0
for precinct in precincts['features']:
  precinct['precinct']=precinct['properties']['precinct']
  polygons.append([shape(precinct['geometry']),precinct['properties']['precinct']])

long=0
lat=0
for p in polygons:
  cord=p[0].centroid.coords[0]
  long+=cord[0]
  lat+=cord[1]
long/=len(polygons)
lat/=len(polygons)
with open('./boston.json','w') as file:
  json.dump({
    "latitude": lat, 
    "longitude": long,
  },file)
# precincts['features']
# polygons[0][0].centroid.coords[0]

In [78]:
df=pd.read_csv('./first-phase/Boston/Boston Db.csv')
#prev_crime_data=pd.read_csv('./seattle-precinct-wise-dataset.csv')
df=df[0:50000]
list1=[]
for polygon in polygons:
  c=0
  count={'precinct': polygon[1],'total':0}
  # count=0
  for index,row in df.iterrows():
    if polygon[0].contains(Point(row['Longitude'],row['Latitude'])):
      if row['Offence'] not in count:
        count[row['Offence']]=0
      count[row['Offence']]+=1
      count['total']+=1
  list1.append(count)

In [79]:
# list1
crime_data=pd.DataFrame.from_dict(list1)
#prev_crime_data=pd.read_csv('./nyc-precinct-wise-dataset.csv')
#prev_crime_data=prev_crime_data.iloc[:,1:]
#print(prev_crime_data.head())
print(crime_data.head())
# crime_data.to_csv('wdc-precinct-wise-dataset.csv')

  precinct  total  Theft  Others  Accident  Murder  Fraud  Robbery  \
0     0113    143   12.0    67.0      26.0     1.0    6.0      1.0   
1     0102     71    5.0    39.0       5.0     NaN    2.0      1.0   
2     0105    131   11.0    77.0      12.0     3.0    1.0      3.0   
3     0203     41    4.0    21.0       2.0     NaN    NaN      2.0   
4     0205     36    8.0    19.0       1.0     1.0    NaN      NaN   

   Public Nuisance  Assault  Arson  Drugs  Missing  Sex Crime  Gun Violence  
0             16.0     12.0    1.0    1.0      NaN        NaN           NaN  
1              9.0     10.0    NaN    NaN      NaN        NaN           NaN  
2             12.0      7.0    2.0    NaN      3.0        NaN           NaN  
3             10.0      2.0    NaN    NaN      NaN        NaN           NaN  
4              5.0      1.0    1.0    NaN      NaN        NaN           NaN  


In [80]:
precincts=crime_data.iloc[:,0]
print(precincts)
temp1=crime_data.iloc[:,1:].fillna(0)
#temp2=prev_crime_data.iloc[:,1:].fillna(0)
print(temp1.head())
#print(temp2.head())
#x=temp1.add(temp2)
#x['precinct']=precincts
# x
crime_data.to_csv('boston-precinct-wise-dataset.csv')

0       0113
1       0102
2       0105
3       0203
4       0205
       ...  
250     1818
251    0502A
252     0115
253     0104
254     2002
Name: precinct, Length: 255, dtype: object
   total  Theft  Others  Accident  Murder  Fraud  Robbery  Public Nuisance  \
0    143   12.0    67.0      26.0     1.0    6.0      1.0             16.0   
1     71    5.0    39.0       5.0     0.0    2.0      1.0              9.0   
2    131   11.0    77.0      12.0     3.0    1.0      3.0             12.0   
3     41    4.0    21.0       2.0     0.0    0.0      2.0             10.0   
4     36    8.0    19.0       1.0     1.0    0.0      0.0              5.0   

   Assault  Arson  Drugs  Missing  Sex Crime  Gun Violence  
0     12.0    1.0    1.0      0.0        0.0           0.0  
1     10.0    0.0    0.0      0.0        0.0           0.0  
2      7.0    2.0    0.0      3.0        0.0           0.0  
3      2.0    0.0    0.0      0.0        0.0           0.0  
4      1.0    1.0    0.0      0.0       

In [5]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default='chrome'
crime_data=pd.read_csv('seattle-precinct-wise-dataset.csv')
data=json.load(open('./seattle.json'))
lat=data['latitude']
long=data['longitude']

In [6]:
fig=px.choropleth_mapbox(crime_data,locations='precinct',featureidkey="properties.precinct",geojson=precincts,color='total',color_continuous_scale='ylorrd')
# fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(mapbox_style="carto-positron", 
                  mapbox_zoom=9.5,
                  mapbox_center={"lat": lat, "lon": long},
                  margin={"r":0,"t":0,"l":0,"b":0},
                  uirevision='constant')
fig.write_html('first_figure-3.html')